<a href="https://colab.research.google.com/github/RileyWClarke/BiancoGroup/blob/master/fftfilter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


#Potentially useful astropy stuff
import astropy
import astropy.io.ascii as ascii
from astropy.table import Table
from astropy.io import fits
from astropy.stats import LombScargle
from astropy.stats import sigma_clipped_stats
from astropy.stats import sigma_clip
from astropy.modeling import powerlaws
from astropy import constants as const
from astropy import units as u

#For reading in and organizing data
import pandas as pd
import requests
import json

#Misc
import scipy
from scipy import stats
from scipy.optimize import curve_fit
from collections import Counter
import itertools


#Making Figures look nice
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 18

In [0]:
filenames = []

for i in range(1000):
    if i<10:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo20613500'
                         +str(i)+'-c03_llc.fits')
    elif i<100:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo2061350'
                         +str(i)+'-c03_llc.fits')
    else:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135'
                         +str(i)+'-c03_llc.fits')

In [3]:
d = {}
for j, filename in enumerate(filenames):
    try:
        with fits.open(filename, mode="readonly") as hdulist:
            d["k2bjds{0}".format(j)] = hdulist[1].data['TIME'] 
            d["pdcsap_flux{0}".format(j)] = hdulist[1].data['PDCSAP_FLUX']
            d["pdcsap_flux_err{0}".format(j)] = hdulist[1].data['PDCSAP_FLUX_ERR']
    except: 
        print('no file for loop '+str(j))

no file for loop 0
no file for loop 1
no file for loop 2
no file for loop 3
no file for loop 4
no file for loop 5
no file for loop 6
no file for loop 7
no file for loop 8
no file for loop 9
no file for loop 10
no file for loop 12
no file for loop 13
no file for loop 14
no file for loop 15
no file for loop 16
no file for loop 17
no file for loop 18
no file for loop 19
no file for loop 20
no file for loop 21
no file for loop 22
no file for loop 23
no file for loop 24
no file for loop 25
no file for loop 26
no file for loop 27
no file for loop 28
no file for loop 29
no file for loop 30
no file for loop 31
no file for loop 33
no file for loop 34
no file for loop 35
no file for loop 36
no file for loop 37
no file for loop 38
no file for loop 39
no file for loop 40
no file for loop 41
no file for loop 42
no file for loop 43
no file for loop 44
no file for loop 45
no file for loop 46
no file for loop 47
no file for loop 48
no file for loop 49
no file for loop 50
no file for loop 51
no file fo

In [4]:
d

{'k2bjds11': array([2144.10322285, 2144.12365529, 2144.14408753, ..., 2213.22381809,
        2213.24424969, 2213.26468129]),
 'k2bjds129': array([2144.10275376, 2144.123186  , 2144.14361805, ..., 2213.22314985,
        2213.24358154, 2213.26401323]),
 'k2bjds130': array([2144.10312752, 2144.12355991, 2144.14399211, ..., 2213.22366885,
        2213.24410047, 2213.26453209]),
 'k2bjds170': array([2144.10275121, 2144.12318346, 2144.14361551, ..., 2213.22314659,
        2213.24357828, 2213.26400997]),
 'k2bjds222': array([2144.10279773, 2144.12323   , 2144.14366206, ..., 2213.22320685,
        2213.24363853, 2213.26407022]),
 'k2bjds257': array([2144.10291709, 2144.1233494 , 2144.14378151, ..., 2213.2233699 ,
        2213.24380156, 2213.26423321]),
 'k2bjds267': array([2144.10293541, 2144.12336773, 2144.14379985, ..., 2213.22339537,
        2213.24382702, 2213.26425867]),
 'k2bjds314': array([2144.10313125, 2144.12356365, 2144.14399584, ..., 2213.22367454,
        2213.24410615, 2213.26453

In [5]:
len(d)/3

46.0